# Parsing Debug Notebook

This somewhat messy notebook makes it easier to debug the parser, because we can just rerun the cells needed to set up the debugging process with various parameters.

In [1]:
import json
import math
import re
from pathlib import Path

import boto3
import pandas as pd

from parse_990_textract.bucket import open_df
from parse_990_textract.filing import create_roadmap, extract_from_roadmap
from parse_990_textract.models import BoundingBox, TableExtractor
from parse_990_textract.parse import create_extractors, find_item, find_pages
from parse_990_textract.postprocessing import clean_filing, clean_f_i, clean_f_ii, clean_f_iii, postprocess
from parse_990_textract.setup import load_extractor_df
from parse_990_textract.table import extract_table_data, find_table_pages, create_tablemap
from parse_990_textract.utils import get_coordinate, get_regex, cluster_words, columnize, cluster_x

In [2]:
bucket = boto3.resource("s3").Bucket("s3-ocr-990s-demo")

Setting `VALIDATE_TOP` to `True` will parse all 25 validation PDFs and compare non-Schedule F output to the validation data. If set to `False`, we run the test code for Schedule F instead.

In [3]:
VALIDATE_TOP = False

In [4]:
if VALIDATE_TOP:
    validation_data = pd.read_csv("validation_data.csv", index_col="job_id").fillna("")
    validation_data.head()

In [5]:
extractor_df = load_extractor_df("parse_data/990_extractors.csv")
roadmap_df = load_extractor_df("parse_data/990_roadmap.csv")
schedule_f_tablemap_df = load_extractor_df("parse_data/schedule_f_table_roadmap.csv")
schedule_f_table_extractor_df = pd.read_csv("parse_data/schedule_f_table_extractors.csv")
schedule_f_row_extractor_df = pd.read_csv("parse_data/schedule_f_row_extractors.csv")

In [6]:
PART_I_HEADER = r"\(a\)\s*Region|\(d\)\s*Activities|\(e\)\s*If activity|\(f\)Total expenditures"
PART_II_HEADER = r"\(b\)\s*IRS code|\(c\)\s*Region|\(d\)\s*Purpose|\(f\)\s*Manner|\(h\)\s*Description"
PART_III_HEADER = r"\(b\)\s*Region|\(e\)\s*Manner of cash|\(h\)\s*Method of va"
PART_I_TABLE_NAME = r"Activities per Region"
PART_II_TABLE_NAME = r"Grants to Organizations Outside the United States"
PART_III_TABLE_NAME = r"Grants to Individuals Outside the United States"

In [7]:
filing_rows = []
schedule_f_part_i_rows = []
schedule_f_part_ii_rows = []
schedule_f_part_iii_rows = []
if VALIDATE_TOP:
    values = validation_data.index.values
else:
    values = []

for i, job_id in enumerate(values):
    print(i)
    print(job_id)
    pdf_key = validation_data.at[job_id, "pdf_key"]
    print(pdf_key)
    
    data = open_df(bucket, job_id)
    lines = data.loc[data["BlockType"] == "LINE"]
    words = data.loc[data["BlockType"] == "WORD"]
    page_map = find_pages(lines)
    roadmap = create_roadmap(
        lines, roadmap_df, page_map
    )
    
    row = extract_from_roadmap(
        words, lines, roadmap, extractor_df, page_map
    )
    row = postprocess(row, job_id, pdf_key, clean_filing)
    filing_rows.append(row)
    
    pages = lines.groupby("Page")
    
    part_i_table = extract_table_data(
        pages, lines, words, PART_I_HEADER, PART_I_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_i_table = postprocess(part_i_table, job_id, pdf_key, clean_f_i)
    if part_i_table is not None:
        schedule_f_part_i_rows.append(
            part_i_table
        )
    part_ii_table = extract_table_data(
        pages, lines, words, PART_II_HEADER, PART_II_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_ii_table = postprocess(part_ii_table, job_id, pdf_key, clean_f_ii)
    if part_ii_table is not None:
        schedule_f_part_ii_rows.append(
            part_ii_table
        )
    part_iii_table = extract_table_data(
        pages, lines, words, PART_III_HEADER, PART_III_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_iii_table = postprocess(part_iii_table, job_id, pdf_key, clean_f_iii)
    if part_iii_table is not None:
        schedule_f_part_iii_rows.append(
            part_iii_table
        )

In [8]:
if VALIDATE_TOP:
    output_data = pd.concat(filing_rows).reset_index(drop=True).set_index("job_id")

In [9]:
VALIDATE_TOP and output_data.head()

False

In [10]:
def clean(x):
    x = str(x)
    x = re.sub(r"\.0\b", "", x)
    x = re.sub("\D", "", x)
    return x

In [11]:
def compare_output(to_validate, to_compare, col):
    return pd.DataFrame(
        {
            "extracted": to_validate.loc[col].loc[
                lambda series: series != to_compare.loc[col]
            ],
            "expected": to_compare.loc[col].loc[
                lambda series: series != to_validate.loc[col]
            ],
        }
    )

In [12]:
if VALIDATE_TOP:
    to_compare = validation_data.set_index("pdf_key").applymap(clean)
    to_validate = output_data[validation_data.columns].set_index("pdf_key").applymap(clean)

    for col in to_validate.index:
        validated = compare_output(to_validate, to_compare, col)
        if validated.any().any():
            print(col)
            print(f"{validated.shape[0]} mismatched items.")
            print(validated)
            print("-"*79)

In [13]:
VALIDATE_TOP and pd.concat(schedule_f_part_i_rows).head(50)

False

In [14]:
VALIDATE_TOP and pd.concat(schedule_f_part_i_rows).tail(50)

False

In [15]:
VALIDATE_TOP and pd.concat(schedule_f_part_ii_rows).head(50)

False

In [16]:
VALIDATE_TOP and pd.concat(schedule_f_part_iii_rows).head(50)

False

In [17]:
if VALIDATE_TOP:
    assert False

In [18]:
table_test_df = open_df(bucket, "a159e2c5a20d354802ab9348b930e6ea05da6151617ca50a8fbb22e7cf01db19")

In [19]:
test_lines = table_test_df.loc[
    table_test_df["BlockType"] == "LINE"
]
test_words = table_test_df.loc[
    table_test_df["BlockType"] == "WORD"
]
test_pages = test_lines.groupby("Page")

In [20]:
page_map = find_pages(test_lines)
roadmap = create_roadmap(
    test_lines, roadmap_df, page_map
)

row = extract_from_roadmap(
    test_words, test_lines, roadmap, extractor_df, page_map
)
row = postprocess(row, "foo_2_3_4_5", "bar_2_3_4_5", clean_filing)
row

No match for website in J Website: www /SAMARITAN ORG


field_name,name,address,city,state,zip,website,gross_receipts,year_formation,state_of_domicile,mission,...,activities_per_region_totals_number_of_offices,activities_per_region_totals_number_of_employees,activities_per_region_totals_total_expenditure,total_number_recipient_foreign_orgs_listed_as_charities,total_number_other_recipient_foreign_orgs_entities,job_id,pdf_key,ein,year,filing_id
0,SAMARITAN'S PURSE,PO BOX 3000,BOONE,NC,28607,,414031085,1980,NC,SAMARITAN'S PURSE IS A NONDENOMINATIONAL EVANG...,...,21,3084,229869706,260,,foo_2_3_4_5,bar_2_3_4_5,2,4,2_4


In [21]:
HEADER = PART_I_HEADER
NAME = PART_I_TABLE_NAME

In [22]:
table_pages = find_table_pages(
    test_pages["Text"].agg(lambda words: " ".join(words)),
    HEADER,
)

In [23]:
table_pages

Page
23    23
28    28
Name: Page, dtype: int64

In [42]:
TEST_INDEX = 1
TEST_PAGE = 28

In [43]:
tablemaps = pd.DataFrame(
    {
        "page": table_pages,
        "tablemap": table_pages.map(
            lambda page: create_tablemap(
                test_lines, schedule_f_tablemap_df, page, NAME
            ).dropna()
        ),
    }
)

In [44]:
tablemaps["tablemap"].iloc[TEST_INDEX]

,Top,Left,Top_Default,Left_Default
Item,,,,
(a) Region,0.104014,0.051200,0.05,0.0
(b) Number of offices,0.104052,0.157223,0.05,0.0
(c) Number of employees,0.103983,0.245368,0.05,0.0
(d) Activities conducted,0.111443,0.338118,0.05,0.0
(e) Specific type,0.103908,0.460974,0.05,0.0
(f) Total Expenditures,0.104096,0.614061,0.05,0.0
"Schedule F, Part I, Item 3a",0.449327,0.005955,0.00,0.0
Bottom (d),0.147532,0.360843,0.00,0.0
Bottom Part I,0.449327,0.005955,0.75,0.0


In [45]:
row_extractors = schedule_f_row_extractor_df.loc[
    schedule_f_row_extractor_df["table"] == NAME
]
table_data = schedule_f_table_extractor_df.loc[
    schedule_f_table_extractor_df["table"] == NAME
].iloc[0]

In [46]:
rows = tablemaps.assign(
    extractor=tablemaps["tablemap"].map(
        lambda tablemap: TableExtractor(
            header_top_label=table_data["header_top"],
            top_label=table_data["table_top"],
            bottom_label=table_data["table_bottom"],
            tablemap=tablemap,
            fields=row_extractors["field"],
            field_labels=row_extractors["col_left"]
        )
    )
)

In [47]:
test_extractor = rows["extractor"].iloc[TEST_INDEX]

In [48]:
test_extractor.get_col_spans(test_words, TEST_PAGE)

0                    (0.0, 0.1572228968143463)
1    (0.1572228968143463, 0.24536845088005066)
2    (0.24536845088005066, 0.3252413272857666)
3     (0.3252413272857666, 0.4519631862640381)
4     (0.4519631862640381, 0.6140613555908203)
5                    (0.6140613555908203, 1.0)
dtype: object

In [49]:
test_extractor.field_labels

0             Top Left Corner
1       (b) Number of offices
2     (c) Number of employees
3    (d) Activities conducted
4           (e) Specific type
5      (f) Total Expenditures
Name: col_left, dtype: object

In [50]:
test_extractor.header_top_label

'(b) Number of offices'

In [51]:
extracted = test_extractor.extract_rows(test_words, TEST_PAGE)

In [52]:
extracted.head(25)

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures
0,CENTRAL AMERICA,2,778,PROGRAM SVCS,EMERGREL/MED/OTH,"30,221,362"
1,CENTRAL AMERICA,,,GRANTS,,"17,805,890"
2,EAST ASIA AND PACIFIC,5,165,PROGRAM SVCS,COMMDEV/RECONST/OTH,"3,558,040"
3,EAST ASIA AND PACIFIC,,,GRANTS,,"17,177,556"
4,EUROPE,2,23,PROGRAM SVCS,CHILDMIN/COMMDEV/OTH,"312,868"
5,EUROPE,,,GRANTS,,"2,152,984"
6,MIDDLE EAST,1,12,GRAM SVCS,MED/CHILDMIN/OTH,"362,462"
7,MIDDLE EAST,,,GRANTS,,"7,297,708"
8,NORTH AMERICA,1,32,PROGRAM SVCS,RECONST/COMMDEV/OTH,"894,808"
9,NORTH AMERICA,,,GRANTS,,"19,087,880"


In [53]:
table_words = test_extractor.get_table_words(test_words, TEST_PAGE)

In [54]:
word_clusters = cluster_words(table_words, table_words["Height"].min(), "Midpoint_Y")
[" ".join(word.sort_values(by="Left")["Text"].values) for word in word_clusters]

['CENTRAL AMERICA 2 778 PROGRAM SVCS EMERGREL/MED/OTH 30,221,362',
 'CENTRAL AMERICA GRANTS 17,805,890',
 'EAST ASIA AND PACIFIC 5 165 PROGRAM SVCS COMMDEV/RECONST/OTH 3,558,040',
 'EAST ASIA AND PACIFIC GRANTS 17,177,556',
 'EUROPE 2 23 PROGRAM SVCS CHILDMIN/COMMDEV/OTH 312,868',
 'EUROPE GRANTS 2,152,984',
 'MIDDLE EAST 1 12 GRAM SVCS MED/CHILDMIN/OTH 362,462',
 'MIDDLE EAST GRANTS 7,297,708',
 'NORTH AMERICA 1 32 PROGRAM SVCS RECONST/COMMDEV/OTH 894,808',
 'NORTH AMERICA GRANTS 19,087,880',
 'RUSSIA PROGRAM SVCS CHILDMIN/OTH 46,573',
 'RUSSIA GRANTS 17,481,021',
 'SOUTH AMERICA 2 95 PROGRAM SVCS COMMDEV/CHILDMIN/OTH 2,596,439',
 'SOUTH AMERICA GRANTS 14,375,498',
 'SOUTH ASIA 1 3 PROGRAM SVCS EMERGREL/MED/OTH 1,418,556',
 'SOUTH ASIA GRANTS 4,458,981',
 'SUB-SAHARAN AFRICA 7 1,976 PRO GRAM SVCS COMMDEV/CHURCH/OTH 34,456,727',
 'SUB-SAHARAN AFRICA GRANTS 56,164,353']

In [55]:
word_clusters

[                                     BlockType  Confidence  \
 2758d923-1d54-4e1a-97d3-fb1a264f100b      WORD   97.577690   
 0c4bd9ed-1614-4a12-bb25-2b805f0767b2      WORD   95.575890   
 8818540d-d49b-4873-9541-afce643cbfe5      WORD   99.470100   
 5e847fd6-9ff1-4b53-9706-d2ef237a71d6      WORD   99.458183   
 25495886-9ba2-4521-a029-7a0fa0a86911      WORD   98.742798   
 5fa2d6f4-8d5d-41bb-9809-e6b01d5d02bf      WORD   91.207367   
 0a088942-bc75-404a-92f8-172f0d480301      WORD   97.853508   
 63b2529b-563c-4cb8-9907-07307c3ce5d6      WORD   99.685661   
 
                                                                                Geometry  \
 2758d923-1d54-4e1a-97d3-fb1a264f100b  {'BoundingBox': {'Height': 0.00546364393085241...   
 0c4bd9ed-1614-4a12-bb25-2b805f0767b2  {'BoundingBox': {'Height': 0.00532262586057186...   
 8818540d-d49b-4873-9541-afce643cbfe5  {'BoundingBox': {'Height': 0.00556000974029302...   
 5e847fd6-9ff1-4b53-9706-d2ef237a71d6  {'BoundingBox': {'Height

In [56]:
def columnize(word_cluster, col_spans):
    return col_spans.map(
        lambda span: word_cluster.loc[
            (word_cluster["Right"].between(*span, inclusive="right"))
        ]
    )


def get_cluster_coords(cluster):
    cluster_coords = {
        "Left": cluster["Left"].min(),
        "Right": cluster["Right"].max(),
        "Height": cluster["Height"].max(),
        "Midpoint_X": cluster["Midpoint_X"].median(),
        "Midpoint_Y": cluster["Midpoint_Y"].median(),
        "Top": cluster["Top"].min(),
        "Bottom": cluster["Bottom"].min(),
    }
    cluster_coords["Width"] = cluster_coords["Right"] - cluster_coords["Left"]
    return cluster_coords


def combine_row(row):
    return pd.Series([
        line.map(
            lambda x: x.sort_values(
                by="Left"
            ).reset_index(drop=True)["Text"].fillna("")
        ).agg(
            lambda x: " ".join(x.values)
        ) + " "
        for line in row
    ]).sum().str.strip()

col_spans = test_extractor.get_col_spans(test_words, TEST_PAGE)

col_spans

0                    (0.0, 0.1572228968143463)
1    (0.1572228968143463, 0.24536845088005066)
2    (0.24536845088005066, 0.3252413272857666)
3     (0.3252413272857666, 0.4519631862640381)
4     (0.4519631862640381, 0.6140613555908203)
5                    (0.6140613555908203, 1.0)
dtype: object

Row break scenarios:
1. Previous cluster and current cluster both have entries in the same numeric column
2. Alignment is TOP and current cluster has non-empty columns that are empty in previous row
3. Alignment is BOTTOM and current cluster has empty columns that are non-empty in previous row

In [57]:
y_tol = table_words["Height"].median()
columnized = columnize(word_clusters[0], col_spans)
columnized.index = test_extractor.fields
last_col_coords = pd.DataFrame.from_records(
    columnized.map(
        get_cluster_coords
    )
)
rows = []
if NAME == PART_I_TABLE_NAME:
    numeric_cols = (1,2,5)
elif NAME == PART_II_TABLE_NAME:
    numeric_cols = (4,6)
elif NAME == PART_III_TABLE_NAME:
    numeric_cols = (2,3,5)
current_row = [columnized]
top_ws = (
    last_col_coords["Top"].min()
    - test_extractor.get_table_top(test_words, TEST_PAGE)
)
print(f"Y tolerance: {y_tol}")
print(f"Top whitespace: {top_ws}")
if top_ws > y_tol * 4:
    alignment = "BOTTOM"
else:
    alignment = "UNKNOWN"
print(f"Alignment: {alignment}")
print("First cluster:")
print(" ".join(word_clusters[0].sort_values(by="Left")["Text"].values))
for count, cluster in enumerate(word_clusters[1:]):
    print("-"*50)
    print(f"Alignment: {alignment}")
    print("Cluster:", " ".join(cluster.sort_values(by="Left")["Text"].values))
    columnized = columnize(cluster, col_spans)
    columnized.index = test_extractor.fields
    col_coords = pd.DataFrame.from_records(columnized.map(get_cluster_coords))
    nonempty = col_coords.dropna().index.to_series()
    last_nonempty = last_col_coords.dropna().index.to_series()
    print("Nonempty\n", nonempty)
    print("Last nonempty\n", last_nonempty)
    # more_cols true if current row has non-empty cells that
    # are empty in the preceding row
    more_cols = (~nonempty.isin(last_nonempty)).any()
    print("More cols:", more_cols)
    # less_cols true if last row has non-empty cells that
    # are empty in current row
    less_cols = (~last_nonempty.isin(nonempty)).any()
    print("Less cols:", less_cols)
    # both_numeric true if both rows have entries in numeric cols
    both_numeric = (
        nonempty.isin(numeric_cols) 
        & last_nonempty.isin(numeric_cols)
    ).any()
    y_delta = (
        col_coords["Top"].min()
        - last_col_coords["Bottom"].max()
    )
    print("Y delta", y_delta)
    print("Both numeric:", both_numeric)
    if (
        both_numeric
        or (more_cols and (alignment== "TOP"))
        or (less_cols and (alignment == "BOTTOM"))
        or (y_delta > y_tol)
    ):
        combined_row = combine_row(current_row)
        print(combined_row)
        rows.append(combined_row)
        current_row = [columnized]
    elif less_cols and (alignment == "UNKNOWN"):
        alignment = "TOP"
        current_row.append(columnized)
    elif more_cols and (alignment == "UNKNOWN"):
        alignment = "BOTTOM"
        current_row.append(columnized)
    else:
        current_row.append(columnized)
    last_col_coords = col_coords            

Y tolerance: 0.005322625860571861
Top whitespace: 0.005191856063902378
Alignment: UNKNOWN
First cluster:
CENTRAL AMERICA 2 778 PROGRAM SVCS EMERGREL/MED/OTH 30,221,362
--------------------------------------------------
Alignment: UNKNOWN
Cluster: CENTRAL AMERICA GRANTS 17,805,890
Nonempty
 0    0
3    3
5    5
dtype: int64
Last nonempty
 0    0
1    1
2    2
3    3
4    4
5    5
dtype: int64
More cols: False
Less cols: True
Y delta 0.008711889386177063
Both numeric: True
field
region                     CENTRAL AMERICA
number_offices                           2
number_employees                       778
activities_conducted          PROGRAM SVCS
specific_type_activity    EMERGREL/MED/OTH
total_expenditures              30,221,362
dtype: object
--------------------------------------------------
Alignment: UNKNOWN
Cluster: EAST ASIA AND PACIFIC 5 165 PROGRAM SVCS COMMDEV/RECONST/OTH 3,558,040
Nonempty
 0    0
1    1
2    2
3    3
4    4
5    5
dtype: int64
Last nonempty
 0    0
3    3
5 

In [40]:
header_words = test_extractor.get_header_words(test_words, TEST_PAGE)

In [41]:
header_words.sort_values(by="Left")[["Text", "Left", "Right", "Midpoint_Y"]].tail(50)

,Text,Left,Right,Midpoint_Y
Id,,,,
3aba1565-0e4d-435a-84dc-d7940aa058e2,independent,0.363724,0.426177,0.390066
e63ba199-d60f-4f68-af28-f9be445d0230,contractors,0.366943,0.423277,0.401127
ad357a58-e8b4-44df-bbed-0740f073778e,Number,0.378422,0.419423,0.354889
508903dd-641f-4b8e-9e23-4e84b9051843,in,0.384696,0.393905,0.378326
e32235e4-f49d-489b-b4a5-50f4fdbaf54d,region,0.397372,0.428716,0.379397
34c1e4c3-82e1-4b33-98ad-47dadd7f0c69,or,0.418680,0.431924,0.367073
4f5338ae-c010-4c8e-a2b0-7d8b0fe08dfb,of,0.421998,0.432811,0.354679
c9d05bf7-2bc9-4027-a17f-03e51631ae49,or,0.432079,0.442744,0.378613
9c7c484c-ea3f-488f-933e-ba0dabee2a25,"services,",0.445761,0.489507,0.390754
